In [83]:
#imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures

In [84]:
#reading in dataset train 
df = pd.read_csv('./datasets/train_cleaned.csv')
df_test = pd.read_csv('datasets/test_cleanded.csv')

## Standar Scaling 

In [85]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2023)


In [86]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

In [87]:
sc = StandardScaler()
Z_train = sc.fit_transform(X_train) 
Z_test = sc.transform(X_test)
Z_test_val=sc.transform(df_test)

In [88]:
df_test.head()

,Unnamed: 0,Id,PID,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,Bsmt Unf SF,...,Garage Cond_Fa,Garage Cond_TA,Paved Drive_N,Paved Drive_Y,Kitchen AbvGr_1,Kitchen AbvGr_2,Fence_MnPrv,Sale Type_COD,Sale Type_New,Sale Type_WD
0,0,2658,902301120,69.000000,9142,1910,1950,0.0,0,1020,...,0,0,0,1,0,1,0,0,0,1
1,1,2718,905108090,69.545961,9662,1977,1977,0.0,0,1967,...,0,1,0,1,0,1,0,0,0,1
2,2,2414,528218130,58.000000,17104,2006,2006,0.0,554,100,...,0,1,0,1,1,0,0,0,1,0
3,3,1989,902207150,60.000000,8520,1923,2006,0.0,0,968,...,0,1,1,0,1,0,0,0,0,1
4,4,625,535105100,69.545961,9500,1963,1963,247.0,609,785,...,0,1,0,1,1,0,0,0,0,1


## Lasso

In [89]:
lasso_alphas = np.linspace(300, 400, 100) 
lasso_cv = LassoCV(alphas = lasso_alphas, cv=10, max_iter=100000)

In [90]:
lasso_cv.fit(Z_train, y_train);

In [91]:
lasso_cv.alpha_

374.74747474747477

In [92]:
np.mean(cross_val_score(lasso_cv, X, y, cv=5))

0.8983433557819633

In [93]:

print(lasso_cv.score(Z_train, y_train))
print(lasso_cv.score(Z_test, y_test))

0.9261307963976271
0.9242547617419916


In [94]:
lasso_cv.fit(X, y)
df_test['SalePrice'] = lasso_cv.predict(Z_test_val)

C:\Users\seefe\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LassoCV was fitted with feature names
  warnings.warn(


In [95]:
submission = df_test[['Id', 'SalePrice']]
submission.set_index('Id', inplace =True) 
submission.to_csv('Submissions/subl_lasso.csv')